# WARNING! 
### This notebook will get you banned by Google after some usage. This is because (despite 90% of my results being scraped off of public government websites) Google cannot differentiate between a thorough job search and a competitor, and once this script uses your built database to make calls they will disable your API key, usually with a temporary ban first.
This notebook got me 4 interviews and they were all for web hacking jobs I was not interested in.

In [ ]:
import re
import typer
import googlemaps
import time
import pandas as pd
import inspect
#import sys
import requests
import scrapy
from scrapy_splash import SplashRequest
import scrapy_splash
import json


In [ ]:

def objinfo(obj):
    for i in dir(obj):
       print(i, getattr(obj, i))


In [ ]:

def defaults(defawlt):
    #defult = input("Please enter a value for {}: ".format(defawlt))
    #try:
    #    defult = int(defult)
    #finally:
    #    return defult
    return input("Please enter a value for {}: ".format(defawlt))


In [ ]:

def locations(df, key):
    (sites, failures) = list_websites(df, key)
    #print(sites)
    #print(failures)
    final_locations = []
    for i in sites:
        if i[0] != None:
            uri = find_landing(i[0])
            print("At {}, we found {}".format(df['name'][i[2]],"{}{}".format(uri[0],uri[1])))
            final_locations.append(uri[1])
    return final_locations


In [ ]:

def rate_limiting(rate):
    print("Got rate limited, chillin for a bit")
    time.sleep(1+rate)
    print("Ok back to work")


In [ ]:

def find_landing(site): #currently fails with some dynamic sites
    landingsfile = open('joblandings.txt', 'r')
    landingslist = landingsfile.read().split('\n')
    for i in landingslist:
        #print(i, site)
        try:
            #print("trying {}".format("{}{}".format(site, i)))
            response = requests.head("{}{}".format(site, i))
            #print("got {}".format(response.status_code))
            while response.status_code == 429:
                i = 0
                rate_limiting(i)
                response = requests.head("{}{}".format(site, i))
                i += 2
                if i >= 20:
                    break
            if response.status_code >= 300 and response.status_code < 400:
                response = requests.head(re.sub(r"http:\/\/","https://","{}{}".format(site, i)))
                
        except requests.ConnectionError as e:
            #print("Connection error: {}".format(e))
            pass
        
        if response.status_code == 200:
            return ("job landing: ","{}{}".format(site, i))
    return ("no job landing, just ","{}".format(site))


In [ ]:

def list_websites(df, key):
    sites = []
    failures = []
    for i in range(0, len(df)):
            print(i)
            sites.append((json.loads()["result"].get("website"), df['place_id'][i], i))
            if sites[-1] == None:
                print("Location {} does not have a listed website, adding to failures.".format(df['name'][i]))
                failures.append(df['place_id'][i])
                sites.pop()
    return (sites, failures)


In [ ]:

def crawl(url):
    #res = yield requests.get(url)
    #print(res)
    print("test")
    yield SplashRequest(url, self.parse_result,
    args={
        # optional; parameters passed to Splash HTTP API
        'wait': 0.5,

        # 'url' is prefilled from request url
        # 'http_method' is set to 'POST' for POST requests
        # 'body' is set to request body for POST requests
    },
    endpoint='render.json', # optional; default is render.html
    splash_url='<url>',     # optional; overrides SPLASH_URL
    slot_policy=scrapy_splash.SlotPolicy.PER_DOMAIN,  # optional
    )


In [ ]:

def retrieve_website(key, placeid):
    #currently requests too many fields that are not used
    url = "https://maps.googleapis.com/maps/api/place/details/json?place_id={}&fields=name%2Crating%2Cwebsite&key={}".format(placeid, key)
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    return response


In [ ]:

def places_crawl(key, geo, search_string, radius):
    radius = int(radius)
    (lat, lng) = map(geo[0]['geometry']['location'].get, ('lat', 'lng'))
    theplaces = []
    for i in range(-radius, radius):
        for j in range(-radius, radius):
            t = lat + i*(1/69)
            g = lng + j*(1/69)
            place = places(key, (t,g), search_string, 1.41421356236)
            if place.empty:
                pass
            else: 
                #pd.concat([theplaces, place], axis=0)
                theplaces.append(place)
                #print(place)
                print(theplaces)
    return theplaces



In [ ]:

def places(key, geo, search_string, radius):
    (lat, lng) = geo
    gmaps = googlemaps.Client(key)
    response = gmaps.places_nearby(
            location=(lat, lng),
            keyword=search_string,
            radius=int(radius) * 1_609.344
    )
    places = []
    places.extend(response.get('results'))
    token = response.get('next_page_token')
    while token:
        time.sleep(2)
        response = gmaps.places_nearby(
            location=(lat, lng),
            keyword=search_string,
            radius=int(radius) * 1_609.344,
            page_token=token
        )
        places.extend(response.get('results'))
        token = response.get('next_page_token')
    df = pd.DataFrame(places)
    #df['url'] = 'https://www.google.com/maps/place/?q=place_id:' + df['place_id']
    #df.to_excel('{0}.xlsx'.format(search_string), index=False)
    #return pd.read_excel('{0}.xlsx'.format(search_string))
    return df


In [ ]:

def main(startpoint: str="", radius: int=0, search_string: str="", key: str=""):
    args = vars()
    typer.echo(f"Hello! Would you like a job?")

    argspec=inspect.getargvalues(inspect.currentframe())
    for arg in argspec.args:
        if args[arg] == "" or args[arg] == 0:
            args[arg] = defaults(arg)
    gmaps = googlemaps.Client(args["key"])
    #geo = gmaps.geocode(address=args["startpoint"])
    #df2 = places_crawl(args["key"], geo, args["search_string"], args["radius"])
    
    #print(df2)
    #df = pd.concat(df2, axis=0, ignore_index=True)
    df = pd.read_excel('no.xlsx')
    df = df.drop_duplicates(subset=['place_id'])
    df['url'] = 'https://www.google.com/maps/place/?q=place_id:' + df['place_id']
    df.to_excel('{0}.xlsx'.format(args["search_string"]), index=False)
    print(df)
    #print(df2)
    #df = pd.read_excel('tmp.xlsx')
    #final_locations = []
    #for i in df2:
    #    final_locations += locations(i, args["key"])
    final_locations = locations(df, args["key"])
    print("\n\nHere is your to-dos: ")
    for i in [*set(final_locations)]:
        print(i)


In [ ]:
if __name__ == "__main__":
    typer.run(main)